In [1]:
import csv

In [2]:
#!!! date_id manca in date.csv e tourney.csv !!!

In [3]:
rows = []

with open('tennis.csv', 'r') as file:
    data = csv.DictReader(file, delimiter=',')
    for row in data:
        rows.append(row)

In [4]:
for row in rows:
    row['match_id'] = row['match_num'] + '_' + row['tourney_id']

# Match

In [5]:
match_feat = ['tourney_id', 'match_id', 'winner_id', 'loser_id', 'score', 'best_of', 'round', 'minutes', 'w_ace', 
              'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 
              'l_ace',   'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 
              'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points']

In [6]:
match_rows = []

for row in rows:
    match_row = {k: row[k] for k in match_feat}
    match_rows.append(match_row)

# Players

In [7]:
male_rows = []

with open('male_players.csv', 'r') as file:
    male_data = csv.reader(file, delimiter=',')
    for row in male_data:
        male_rows.append(row)
        
male_players = []

for row in male_rows[1:]:
    male_players.append(row[0] + " " + row[1])
    


In [8]:
player_birth = dict()
player_ht = dict()
player_rows = []
processed_id = []


#primo ciclo for per:
#1) creare nei dizionari i valori con i player_id e le liste vuote
#2) riempire tali liste con tutti le ht e tutti gli year of birth
for row in rows:
    player_id = row['winner_id']
    if player_id not in player_birth.keys():
        player_birth[player_id] = []
    if player_id not in player_ht.keys():
        player_ht[player_id] = []
#age ha dei missing values. In caso di missing non appendiamo niente
    if len(row['winner_age']) > 0:
        player_birth[player_id].append(int(row['tourney_date'][0:4]) - int(float(row['winner_age'])))
    player_ht[player_id].append(row['winner_ht'])   
    
#ripetiamo per i loser
    player_id = row['loser_id']
    if player_id not in player_birth.keys():
        player_birth[player_id] = []
    if player_id not in player_ht.keys():
        player_ht[player_id] = []
    if len(row['loser_age']) > 0:
        player_birth[player_id].append(int(row['tourney_date'][0:4]) - int(float(row['loser_age'])))
    player_ht[player_id].append(row['loser_ht'])   

for row in rows:
    player_id = row['winner_id']     
#evitiamo di processare più volte registrando ogni id in un'apposita lista
    if player_id not in processed_id:
        processed_id.append(player_id)
        player_row = dict()
        player_row['player_id'] = player_id
        player_row['country_id'] = row['winner_ioc']
        player_row['name'] = row['winner_name']
        if row['winner_name'] in male_players:
            player_row['sex'] = 'M'
        else:
            player_row['sex'] = 'F'
#prendiamo l'altezza massima dalla rispettiva lista (creata nel precedente ciclo for)
        player_row['ht'] = max(player_ht[player_id])
        player_row['hand'] = row['winner_hand']
#prendiamo l'anno di nascita minimo dalla rispettiva lista
#se la lista ha len 0, quell'id aveva solo missing values come valori 
        if len(player_birth[player_id]) > 0:
            player_row['year_of_birth'] = min(player_birth[player_id])
        else:
            player_row['year_of_birth'] = ''
        player_rows.append(player_row)
        
#ripetiamo per i loser
#grazie alla lista processed_id evitiamo di processare lo stesso id come winner e come loser
    player_id = row['loser_id']     
    if player_id not in processed_id:
        processed_id.append(player_id)
        player_row = dict()
        player_row['player_id'] = player_id
        player_row['country_id'] = row['loser_ioc']
        player_row['name'] = row['loser_name']
        if row['loser_name'] in male_players:
            player_row['sex'] = 'M'
        else:
            player_row['sex'] = 'F'
        player_row['ht'] = max(player_ht[player_id])
        player_row['hand'] = row['loser_hand']
        if len(player_birth[player_id]) > 0:
            player_row['year_of_birth'] = min(player_birth[player_id])
        else:
            player_row['year_of_birth'] = ''
        player_rows.append(player_row)

#  Tourney

In [9]:
tourney_feat = ['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_spectators', 
                'tourney_revenue']

In [10]:
tourney_rows = []

for row in rows:
    tourney_row = {k: row[k] for k in tourney_feat}
    tourney_rows.append(tourney_row)

# Date

In [11]:
#age ha formato: YYYYMMMDD

date_rows = []

for row in rows:
    date_row = dict()
    date_row['day'] = int(row['tourney_date'][6:])
    date_row['month'] = int(row['tourney_date'][4:6])
    date_row['year'] = int(row['tourney_date'][0:4])
    if date_row['month'] in [1, 2, 3]:
        date_row['quarter'] = 1
    elif date_row['month'] in [4, 5, 6]:
        date_row['quarter'] = 2
    elif date_row['month'] in [7, 8, 9]:
        date_row['quarter'] = 3
    else:
        date_row['quarter'] = 4
    date_rows.append(date_row)


# Geography

In [12]:
#lista finale
geo_rows = []

#lista delle row del country csv
ioc_rows = []

with open('country-codes.csv', 'r') as file:
    ioc_data = csv.DictReader(file, delimiter=',')
    for row in ioc_data:
        ioc_rows.append(row)
        
#lista di tutti gli ioc unici dei nostri player
ioc_list = []

for row in player_rows:
    ioc_list.append(row['country_id'])
    
ioc_list = list(set(ioc_list))

In [13]:
#ciclichiamo le row del country csv
#se l'ioc è presente nella lista dei nostri ioc, prendiamo le feature di quella row che ci interessano 
for row in ioc_rows:
    if row['IOC'] in ioc_list:
        geo_row = dict()
        geo_row['country_ioc'] = row['IOC']
        geo_row['continent'] = row['Continent']
        geo_row['language'] = row['Languages']
        geo_rows.append(geo_row)

# Export

In [15]:
with open('player.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, 
                        fieldnames=player_rows[0].keys(),

                       )
    fc.writeheader()
    fc.writerows(player_rows)
    
with open('match.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, 
                        fieldnames=match_rows[0].keys(),

                       )
    fc.writeheader()
    fc.writerows(match_rows)

with open('tourney.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, 
                        fieldnames=tourney_rows[0].keys(),

                       )
    fc.writeheader()
    fc.writerows(tourney_rows)
    
with open('date.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, 
                        fieldnames=date_rows[0].keys(),

                       )
    fc.writeheader()
    fc.writerows(date_rows)
    
with open('geography.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, 
                        fieldnames=geo_rows[0].keys(),

                       )
    fc.writeheader()
    fc.writerows(geo_rows)